# AI Summer School 2022 - Challenge Total Energies 
# Wind turbine production forecast

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from pathlib import Path
from lightgbm import LGBMRegressor


from challenge.preprocessing import build_new_features, build_polynomial_wind_features

%matplotlib inline

In [ ]:
VERBOSE = True
EXPORT = False

## 1. Load data

In [ ]:
current_path = Path.cwd()
data_path = current_path / "data"
output_path = current_path / "outputs"
output_path.mkdir(parents=True, exist_ok=True)

In [ ]:
data = pd.read_csv(data_path / "train.csv", parse_dates=["date"], index_col=["date"]).sort_index()

test = pd.read_csv(data_path / "test.csv", parse_dates=["date"], index_col=["date"]).sort_index()

In [ ]:
assert data.index.min() == pd.Timestamp('2009-07-01 01:00:00')
assert data.index.max() == pd.Timestamp('2012-06-26 12:00:00')

assert test.index.min() == pd.Timestamp('2011-01-01 01:00:00')
assert test.index.max() == pd.Timestamp('2012-06-28 12:00:00')

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
test.describe()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
data.asfreq("h").tail(36+48+36+48+36)[["ws"]].plot(ax=ax)
plt.title("Train")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
test.asfreq("h").tail(36+48+36+48+36)[["ws"]].plot(ax=ax)
plt.title("Test")

In [ ]:
expected_data_shape = (18_756, 5)
np.testing.assert_allclose(data.shape, expected_data_shape)

expected_test_shape = (7_488, 4)
np.testing.assert_allclose(test.shape, expected_test_shape)

Then, we check that we have no NA values in the dataframes.

In [ ]:
assert not data.isnull().any(axis=None) and not test.isnull().any(axis=None)

Nice, our data is exactly the types we want, and no NA are present, we can pass on to an exploratory data analysis (EDA).

## 2. EDA and first models

Let's start by checking what we want to predict : the power measurement wp1 of the first farm.

### 2.1. Target variable `Wp1`: the normalized power generated by one wind farm 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
data[["wp1"]].head(300).plot(ax=ax)

In [ ]:
data["wp1"].hist()

In [ ]:
np.testing.assert_allclose(data["wp1"].min(), 0.0)
np.testing.assert_allclose(data["wp1"].max(), 0.947)

### 2.2. Windspeed

The Critical parameter in predicting the wind power, obviously seems to be the wind speed. Let us observe this parameter.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
data[["ws"]].head(300).plot(ax=ax)

Let's see if there is any correlation between power and speed by taking a look at one of them, function of the other.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
data[["ws", "wp1"]].head(300).plot.scatter(ax=ax, x="ws", y="wp1", xlabel="winspeed", ylabel="windpower")


There clearly seems to be a correlation between the two ! When windspeed rises, the wind power rises, on average, even if the relation between the two is not linear. 

We can confirm this by calculating the correlation coeficient betwen the two. Actually we can directly calculate all of the correlation coefficient between all variables in the dataset in one line of code. Let us do so.

In [ ]:
data.corr()

The correlation coefficient between windpseed and wind powe is 0.7 : this is very high indeed, our firt conclusion were true. Let us recall Pearson's definition of correlation, which is the one we used here.

For a sample, it is defined by : 
    

$$
\frac{\sum \limits _{i=1} ^{n} (x_{i} - \bar x) (y_{i} - \bar y)}{\sqrt{\sum \limits _{i=1} ^{n}(x_{i} - \bar x)^{2}}\sqrt{\sum \limits _{i=1} ^{n}(y_{i} - \bar y)^{2}}} 
$$

What is important to recall is that it is comprised in the range $[-1, 1]$ and : 
    - it is equal to 1 if the two variables are exactly the same
    - -1 if the two varables are the exact opposite
    - when it is equal to 0, the two variables have nothing in common : they are independent one from the other, for example this could be the value of the bitcoin and the average windspeed in south korea, we know these two have nothing in common.
    - when it is > 0, the two variables are positively correlated, this means that on average, when one goes up, the other goes up too.
    - when it is < 0, the two variables are negatively correlated, this means that on average, when one goes up, the other goes down.

### 2.2.1. Model `wp1 ~ ws` relationship

Based on this first EDA, a very simple model we can try to predict our sample is to try a linear model : 

$$ wp_{1} = \alpha . ws + \beta$$

We now cut our dataframe in two : one dataframe will be used for training, and the other one will be used to estimate what is the value of this first model we have made. 
For this, why do we not directly use the test set ? The reason is that for the test set, we do not know the exact value of the power measurement.

In [ ]:
data = build_new_features(data)

test = build_new_features(test)

In [ ]:
data

### 2.3. Wind direction: `u` & `v`

Now let's take a look at direction.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
data[["sin_wd"]].head(300).plot(ax=ax)

Interesting to see that, as expected, it is comprised between 0 and 360. And when it crosses 360 it goes to 0, as expected for a direction. But this makes it a highly discontinuous function in 360. How should we treat this in the models ? That's a question for you to answer..

One more thing about direction, the wind vector can be either expressed as :
- windspeed, and wind direction, 
- or u and v components. 

These two representations are interchangeable (for math guys, there is a bijection between these two representations)
And in our case, the convention for the wind direction used in our data is wind vector azimuth. For more information on these matters, please check the following website which explains these representations : 
    http://tornado.sfsu.edu/geosciences/classes/m430/Wind/WindDirection.html


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
data[["u", "v"]].head(300).plot(ax=ax)

In [ ]:
pd.concat([data["u"]**2 + data["v"]**2, data["ws"]**2], axis=1).plot(kind = "scatter", x=0, y="ws")
plt.xlabel("ws^2")
plt.ylabel("u^2 + v^2")

In [ ]:
pd.concat([data["u"], data["ws"] * data["sin_wd"]], axis=1).plot(kind = "scatter", x="u", y=0)
plt.xlabel("u")
plt.ylabel("ws * sin(wd)")

## 3. Next steps Modeling

In [ ]:
df_full = pd.concat([data, test]).sort_index()
df_full["ws_t_1"] = df_full["ws"].shift()
df_full["sin_wd_t_1"] = df_full["sin_wd"].shift()

In [ ]:
df_full

In [ ]:
data = df_full.loc[data.iloc[1:].index]
test = df_full.loc[test.index].drop("wp1", axis=1)

In [ ]:
selected_wind_features = ["ws_t_1", "ws", "sin_wd_t_1", "sin_wd", "u", "v"]
# selected_wind_features = ["u", "v"]
selected_time_features = ["hour", "day", "month"]
selected_features = selected_wind_features + selected_time_features

In [ ]:
polynomial_features = PolynomialFeatures(degree=2, include_bias=False)

In [ ]:
df_polynomial_data = build_polynomial_wind_features(
    polynomial_features, data, selected_wind_features, selected_time_features
)

In [ ]:
data = pd.concat([df_polynomial_data, data["wp1"]], axis=1)

In [ ]:
data

In [ ]:
test = build_polynomial_wind_features(
    polynomial_features, test, selected_wind_features, selected_time_features
)

In [ ]:
selected_features = [col for col in data.columns if col != "wp1"]

## Split train and validation set

In [ ]:
train, validation = train_test_split(data, test_size=0.2, shuffle=True)

In [ ]:
X_train = train.drop("wp1", axis=1)
y_train = train["wp1"]

X_val = validation.drop("wp1", axis=1)
y_val = validation["wp1"]

In [ ]:
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

### Linear regression

In [ ]:
lm = LinearRegression()
lm.fit(X_train[selected_features], y_train)

In [ ]:
print(mean_absolute_error(lm.predict(X_train[selected_features]), y_train))
print(mean_absolute_error(lm.predict(X_val[selected_features]), y_val))

In [ ]:
scores = cross_val_score(lm, data[selected_features], data["wp1"], cv=5, scoring=mae_scorer)
print(scores)
print(-scores.mean(), scores.std())

Nice ! We have our first model and it gives an error of 0.13 !..

Now wait, what is the value of that first model ? How can we know if 0.13 is actually a good error ? Well for this, a very neat way to be able to know if our model is worth anything is to compare it to a naive model. A naive model can be for example to predict everytime the same value, whatever the conditions. One of these naive model we have at hand would be to predict the mean value of the wind power in the train set. Let's see what would this model give. 

### Dummy regression

In [ ]:
dm = DummyRegressor(strategy='mean')
dm.fit(X_train[selected_features], y_train)

In [ ]:
print(mean_absolute_error(dm.predict(X_train[selected_features]), y_train))
print(mean_absolute_error(dm.predict(X_val[selected_features]), y_val))

In [ ]:
dm = DummyRegressor(strategy='median')
dm.fit(X_train[selected_features], y_train)

In [ ]:
print(mean_absolute_error(dm.predict(X_train[selected_features]), y_train))
print(mean_absolute_error(dm.predict(X_val[selected_features]), y_val))

Yes ! Good news, our model did really learn something good ! We are a lot better than the 'mean' or 'median' model, around 30% better, based on this metric.

We have already seen first models above : the linear model with one variable (windspeed), and two naive models (median, and mean). It will be your job from now on to determine the best model, but let's already take a look at one classic model that data scientists usually try on for nearly any subject : Random Forest. 

In [ ]:
rf = RandomForestRegressor(n_jobs=-1, max_depth=4)
rf.fit(X_train[selected_features], y_train)

scores = cross_val_score(rf, data[selected_features], data["wp1"], cv=5, scoring=mae_scorer)
print(-scores)
print(-scores.mean(), scores.std())

In [ ]:
# lgbm = LGBMRegressor()
lgbm = LGBMRegressor(n_estimators=100, max_depth=3)

lgbm.fit(X_train[selected_features], y_train)

scores = cross_val_score(lgbm, data[selected_features], data["wp1"], cv=5, scoring=mae_scorer)
print(-scores)
print(-scores.mean(), scores.std())

Random Forest does only very slightly better than the linear model.

# Predictions on test set

In [ ]:
lgbm.fit(data[selected_features], data["wp1"])

Now our model is fit, we can pass on to the predictions.

_Note: be careful when generating your submission file. Indeed, it needs to be a csv file with ";" as separator._

In [ ]:
X_test[selected_features].columns

In [ ]:
X_test = test.copy()

df_predictions = pd.DataFrame({
    "wp1": lgbm.predict(X_test[selected_features]),
}, index=test.index)

df_predictions.head()

In [ ]:
if EXPORT:
    df_predictions.to_csv(output_path / "predictions.csv", sep=";")

Now it is your turn, what better model can you think of ?